# TORCH.AUTOGRAD를 사용한 자동 미분

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
#w,b는 최적화를 해야 하는 매개변수
#require_grad의 값은 x.requires_grad_(True)메소드를 사용하여 나중에 설정할 수 있음

In [2]:

print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x0000024BC54C28E0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x0000024BC54C2850>


### 변화도(Gradient) 계산하기

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2410, 0.1919, 0.3276],
        [0.2410, 0.1919, 0.3276],
        [0.2410, 0.1919, 0.3276],
        [0.2410, 0.1919, 0.3276],
        [0.2410, 0.1919, 0.3276]])
tensor([0.2410, 0.1919, 0.3276])


### 변화도 추적 멈추기

변화도 추적은 사전 학습된 신경망을 미세조정(fine-tuning)할 때 사용합니다.
또한 순전파 단계만 수행할 때 연산 속도가 향상됩니다. 

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
#위와 동일한 결과
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


### 연산 그래프에 대한 추가 정보

순전파 단계에서 autograd 는 다음 두 가지 작업을 수행
* 요청된 연산을 수행하여 결과 텐서를 계산하고,
* DAG에 연산의 변화도 기능(gradient function)를 유지(maintain)합니다.

역전파 단계는 DAG 뿌리(root)에서 .backward()가 호출될 때 시작됩니다.
* 각 .grad_fn으로부터 변화도를 계산하고,
* 각 텐서의 .grad 속성에 계산 결과를 쌓고(accumulate)
* 연쇄 법칙을 사용하여, 모든 잎(leaf)텐서들까지 전파(propagate)합니다. 